# Erode
first cut is the deepest

In [ ]:
#| default_exp water/erode

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple, deque
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# unique
from treelib import Tree
import heapq


In [ ]:
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))

In [ ]:
#| export


from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, StyleDemo,NamedColor
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground,windy_edge, HexRegion, unique_windy_edge
from HexMagic.terrain import  TerraDemo, Terrain
from HexMagic.terrainpatterns import TerrainPatterns


In [ ]:
#| export
from HexMagic.voronoi import PlateKind
from HexMagic.water.soil import SoilSystem, SoilType

from HexMagic.weather import TerraDemo

In [ ]:
from HexMagic.water.river import River, RiverDemo

In [ ]:
#| export
@dataclass
class ErosionModel:
    """Simple erosion model with age parameter."""
    terrain: 'Terrain'
    soil_system: 'SoilSystem'
    age: float = 1.0  # 0.1 = young/new world, 1.0 = old world
    
    def erode(self, iterations: int = 1, base_rate: float = 5.0, deposition_threshold: float = 50.0):
        """Run erosion simulation.
        
        Args:
            iterations: Number of erosion passes (age scales this)
            base_rate: Base erosion amount per unit stream power
            deposition_threshold: Elevation below which deposition occurs
        """
        effective_iters = max(1, int(iterations * self.age))
        
        for _ in range(effective_iters):
            self._erode_pass(base_rate, deposition_threshold)
    
    
    def _update_soils(self, eroded: np.ndarray, deposited: np.ndarray):
        """Update soil types based on erosion/deposition."""
        soil_types = self.terrain.fields['soil_type']
        elevations = self.terrain.elevations
        
        for i in range(len(elevations)):
            if deposited[i] > 5:
                # Significant deposition → Alluvial
                soil_types[i] = 4
            elif eroded[i] < -20:
                # Heavy erosion → expose bedrock
                if elevations[i] > 500:
                    soil_types[i] = 0  # Granite
                else:
                    soil_types[i] = 1  # Basalt
            elif eroded[i] < -5:
                # Moderate erosion → sandstone/limestone
                soil_types[i] = 3 if soil_types[i] > 2 else 2
        
        # Rebuild regions
        for type_idx in range(len(self.soil_system.types)):
            hexes = set(np.where(soil_types == type_idx)[0])
            self.soil_system.regions[type_idx] = HexRegion(hexes=hexes, hexGrid=self.terrain.hexGrid)


In [ ]:
#| export
@patch
def _find_knickpoints(self: ErosionModel, river: River, gradient_threshold: float = 15.0) -> list[int]:
    """Find knickpoints (sudden gradient changes) in a river.
    
    Returns hex indices where gradient spikes (potential waterfalls).
    """
    gradient = river._calculate_gradient()
    knickpoints = []
    
    # Walk upstream looking for gradient jumps
    for node in river.tree.all_nodes():
        for i, hex_idx in enumerate(node.data):
            if gradient.get(hex_idx, 0) > gradient_threshold:
                knickpoints.append(hex_idx)
    
    return knickpoints

@patch
def _migrate_knickpoint(self: ErosionModel, hex_idx: int, river: River, migration_rate: float = 0.5):
    """Migrate a knickpoint upstream by eroding the lip.
    
    Args:
        hex_idx: Current knickpoint location
        river: The river containing this knickpoint
        migration_rate: How much to erode (scaled by age and soil)
    """
    terrain = self.terrain
    soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
    types = self.soil_system.types
    
    # Get soil resistance at knickpoint
    resistance = types[soil_types[hex_idx]].erosion_resistance
    
    # Find upstream hex(es) - these form the "lip" of the waterfall
    upstream_hexes = []
    for node in river.tree.all_nodes():
        if hex_idx in node.data:
            idx = node.data.index(hex_idx)
            # Upstream is later in the list (remember: outlet is first)
            if idx < len(node.data) - 1:
                upstream_hexes.append(node.data[idx + 1])
            # Also check child nodes (tributaries joining here)
            for child in river.tree.children(node.identifier):
                upstream_hexes.append(child.data[0])
            break
    
    if not upstream_hexes:
        return
    
    # Calculate erosion at the lip
    current_elev = terrain.elevations[hex_idx]
    
    for upstream_idx in upstream_hexes:
        upstream_elev = terrain.elevations[upstream_idx]
        drop = upstream_elev - current_elev
        
        if drop > 5:  # Only erode if there's a real drop
            # Erosion inversely proportional to resistance
            erosion = migration_rate * self.age * drop / (resistance + 0.1)
            erosion = min(erosion, drop * 0.5)  # Don't over-erode
            
            # Erode the upstream lip
            terrain.elevations[upstream_idx] -= erosion
            
            # If we've eroded through soft rock to hard rock, slow down
            if erosion > 10 and soil_types[upstream_idx] in [3, 4]:  # Sandstone/Alluvial
                # Expose harder rock underneath
                soil_types[upstream_idx] = 2  # Limestone (harder)

@patch  
def _erode_pass(self: ErosionModel, base_rate: float, deposition_threshold: float):
    """Single erosion pass with knickpoint migration."""
    terrain = self.terrain
    soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
    types = self.soil_system.types
    
    # Get rivers and compute power
    peaks = terrain.find_peaks(40, 1)
    rivers = [River.from_peak(terrain, p) for p in peaks]
    rivers = [r for r in rivers if r]
    rivers = River.combine_rivers(rivers)
    
    adjustments = np.zeros(len(terrain.elevations))
    deposited = np.zeros(len(terrain.elevations))
    
    for river in rivers:
        flow = river._calculate_flow()
        gradient = river._calculate_gradient()
        
        # === KNICKPOINT MIGRATION ===
        knickpoints = self._find_knickpoints(river, gradient_threshold=12.0)
        for kp in knickpoints:
            self._migrate_knickpoint(kp, river, migration_rate=0.8)
        
        # === FLUVIAL EROSION ===
        for hex_idx in river.hexes:
            power = flow[hex_idx] * gradient[hex_idx]
            
            soil_idx = soil_types[hex_idx]
            resistance = types[soil_idx].erosion_resistance
            
            erosion = base_rate * power * self.age / (resistance + 0.1)
            
            elev = terrain.elevations[hex_idx]
            
            if elev < deposition_threshold and gradient[hex_idx] < 5:
                deposited[hex_idx] += erosion * 0.3
            else:
                adjustments[hex_idx] -= erosion
                
                for neighbor in terrain.hexGrid.neighborsOf(hex_idx):
                    if neighbor >= 0 and terrain.elevations[neighbor] < elev:
                        adjustments[neighbor] -= erosion * 0.3 / (resistance + 0.1)
    
    # Apply changes
    terrain.elevations = np.clip(terrain.elevations + adjustments + deposited, 0, None)
    
    self._update_soils(adjustments, deposited)


In [ ]:
@patch
def demoErosion(self: RiverDemo, age=0.5):
    """Demo erosion with age parameter. age: 0.1=young, 1.0=old"""
    mySize = MapSize(480, 480)
    myBounds = MapRect(MapCord(0, 0), mySize)
    terrain, plates = Terrain.fromSeeds(myBounds,  radius=15)
    terrain.carve_to_ocean(2)
    
    soil = SoilSystem.from_plates(terrain, plates, debug=True)
    
    print(f"\n=== Before erosion (age={age}) ===")
    print(f"Max elev: {terrain.elevations.max():.0f}")
    
    model = ErosionModel(terrain, soil, age=age)
    model.erode(iterations=3, base_rate=3.0)
    
    print(f"\n=== After erosion ===")
    print(f"Max elev: {terrain.elevations.max():.0f}")
    
    # Visualize
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.hexGrid.builder.layers = []
    #terrain.addCoast()
    terrain.hexGrid.builder.adjust("soil", soil.soilOverlay())
    
    return terrain.hexGrid.builder.show()


### Demo

In [ ]:
RiverDemo().demoErosion()

Done at iter 1: 2 lakes

=== SOIL SYSTEM CREATED ===
Total hexes: 1024
Granite     :     0 hexes (  0.0%)
Basalt      :   304 hexes ( 29.7%)
Limestone   :     0 hexes (  0.0%)
Sandstone   :   667 hexes ( 65.1%)
Alluvial    :    53 hexes (  5.2%)

=== Before erosion (age=0.5) ===
Max elev: 472

=== After erosion ===
Max elev: 426


How come no granite or limestone?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good question! Looking at your `SoilSystem.from_plates` logic:

**No Granite** because:
- Granite only appears when `elev > 1000` (high mountains)
- Your max elevation is only ~326m — not nearly high enough

**No Limestone** because:
- Limestone requires mid-elevation AND `precipitation > 750 mm/year`
- Either you don't have a `precipitation` field, or all areas are "dry" (< 750mm)
- The fallback for no precipitation uses plate type → sandstone for continental plates

So your terrain is basically too flat and too dry for those soil types. You could:
1. Increase terrain relief (higher peaks for granite)
2. Add precipitation data (wet areas for limestone)
3. Lower the thresholds in `from_plates`

Which would you like to explore?

Can you build a world that would have interesting soil. these are some helper functions

def generate_plate_terrain(
    bounds, 
    radius=20,
    # Plate generation
    num_plates=10,
    subdivisions=3,
    min_subplate_size=3,
    # Ocean configuration  
    ocean_fraction=0.4,
    oceanic_sides=[],
    edge_factor=1.5,
    # Terrain character
    age='middle',  # 'young', 'middle', 'old'
    formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rolling'
    # Fine-tuning
    elevation_scale=1.0,
    seed=None
):
    """
    Generate terrain based on tectonic plates.
    
    Args:
        bounds: MapRect defining terrain bounds
        radius: Hex radius
        num_plates: Initial number of large plates (for coastlines)
        subdivisions: Depth of plate subdivision for detail
        min_subplate_size: Minimum hex count per subplate
        ocean_fraction: Fraction of initial plates marked oceanic
        oceanic_sides: List of sides ['N','E','S','W'] that are ocean
        edge_factor: How far from edge to mark as oceanic
        age: 'young' (sharp), 'middle', 'old' (eroded)
        formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
        elevation_scale: Multiplier for all elevations
        seed: Random seed
    
    Returns:
        (Terrain, plates) tuple
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Age presets
    age_params = {
        'young':  {'slope': 30, 'variation': 50, 'smoothing': 1, 'peak_height': 200},
        'middle': {'slope': 20, 'variation': 40, 'smoothing': 3, 'peak_height': 150},
        'old':    {'slope': 10, 'variation': 20, 'smoothing': 6, 'peak_height': 80},
    }
    params = age_params.get(age, age_params['middle'])
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Phase 1: Generate coarse plates for coastlines
    coarse_plates = Plate.voronoi(grid, num_plates)
    
    # Mark oceanic plates
    coarse_plates = mark_oceanic_plates(coarse_plates, ocean_fraction=ocean_fraction, seed=seed)
    if oceanic_sides:
        mark_oceanic_by_sides(coarse_plates, grid, factor=edge_factor, oceanic_sides=oceanic_sides)
    
    # Phase 2: Subdivide for detail (preserving plate kind)
    plates = []
    for plate in coarse_plates:
        sub = plate.subdivide(depth=subdivisions, min_size=min_subplate_size)
        plates.extend(sub)
    
    # Calculate ocean distances
    grid.calculate_distances_from_ocean(plates)
    
    # Get boundaries for ridge-based terrain
    boundaries = Plate.boundaries(plates)
    
    # Initialize elevations
    elevations = np.zeros(len(grid.hexes))
    
    # Generate elevations based on formation type
    if formation_type == 'ocean_distance':
        elevations = _elevation_ocean_distance(grid, plates, params, elevation_scale)
    elif formation_type == 'ridge':
        elevations = _elevation_ridge(grid, plates, boundaries, params, elevation_scale)
    elif formation_type == 'volcanic':
        elevations = _elevation_volcanic(grid, plates, params, elevation_scale)
    elif formation_type == 'rolling':
        elevations = _elevation_rolling(grid, plates, params, elevation_scale)
    else:
        raise ValueError(f"Unknown formation_type: {formation_type}")
    
    # Apply smoothing based on age
    for _ in range(params['smoothing']):
        elevations = _smooth_elevations(grid, elevations)
    
    terrain.elevations = elevations
    terrain.colorMap()
    
    return terrain, plates


def _elevation_ocean_distance(grid, plates, params, scale):
    """Original approach: elevation based on distance from ocean."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -100 * scale
            var = params['variation'] * 0.5
        else:
            base = (50 + plate.oceanDistance * params['slope']) * scale
            var = params['variation']
        
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_ridge(grid, plates, boundaries, params, scale):
    """Mountains form at plate boundaries."""
    elevations = np.zeros(len(grid.hexes))
    
    # Build plate lookup
    plate_map = {}
    for plate in plates:
        for idx in plate.hexes:
            plate_map[idx] = plate
    
    for i, hx in enumerate(grid.hexes):
        plate = plate_map.get(i)
        if plate is None:
            continue
        
        pos = grid.index_to_hexposition(i)
        
        # Distance to nearest boundary
        if boundaries:
            dist_to_boundary = min(pos.distance(b) for b in boundaries)
        else:
            dist_to_boundary = 10
        
        if plate.kind == PlateKind.oceanic:
            # Ocean: slight trench near boundaries
            base = -80 * scale
            boundary_effect = -max(0, 2 - dist_to_boundary) * 10 * scale
        else:
            # Continental: ridges at boundaries
            base = 30 * scale
            ridge_height = max(0, 4 - dist_to_boundary) * params['peak_height'] / 4 * scale
            boundary_effect = ridge_height
        
        var = params['variation'] * 0.5
        elevations[i] = base + boundary_effect + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_volcanic(grid, plates, params, scale):
    """Peaks at plate centroids (volcanic islands, mountain cores)."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        centroid = plate.centroid_position()
        
        if plate.kind == PlateKind.oceanic:
            base = -60 * scale
            peak = params['peak_height'] * 0.3 * scale  # Underwater volcanoes
        else:
            base = 40 * scale
            peak = params['peak_height'] * scale
        
        for idx in plate.hexes:
            pos = grid.index_to_hexposition(idx)
            dist = pos.distance(centroid)
            
            # Peak at center, falls off
            height = max(0, peak - dist * params['slope'])
            var = params['variation'] * 0.3
            elevations[idx] = base + height + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_rolling(grid, plates, params, scale):
    """Gentle rolling terrain with minimal features."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -30 * scale
        else:
            base = 20 + plate.oceanDistance * params['slope'] * 0.3
            base *= scale
        
        var = params['variation'] * 0.7
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _smooth_elevations(grid, elevations):
    """Single smoothing pass averaging with neighbors."""
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    return smoothed


Terrain.fromSeeds = generate_plate_terrain

maybe a demo

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's a demo that creates terrain with varied soil types:

```python
def demoInterestingSoil(seed=42):
    """Create terrain with all 5 soil types represented."""
    mySize = MapSize(600, 600)
    myBounds = MapRect(MapCord(0, 0), mySize)
    
    # Key settings for interesting soil:
    # - 'ridge' formation → high mountains at plate boundaries (granite!)
    # - elevation_scale > 1 → push peaks above 1000m
    # - oceanic_sides → clear ocean/land contrast
    terrain, plates = Terrain.fromSeeds(
        myBounds,
        radius=12,
        num_plates=8,
        subdivisions=2,
        age='young',  # Sharp peaks
        formation_type='ridge',  # Mountains at boundaries
        elevation_scale=8.0,  # Push peaks HIGH for granite
        oceanic_sides=['W', 'S'],  # Ocean on west and south
        ocean_fraction=0.3,
        seed=seed
    )
    
    # Add precipitation field for limestone/sandstone differentiation
    # Simple model: more rain on windward (west) side, less inland
    n_hexes = len(terrain.elevations)
    precip = np.zeros(n_hexes)
    
    for i in range(n_hexes):
        pos = terrain.hexGrid.index_to_hexposition(i)
        # Rain decreases as you go east (rain shadow)
        # Also increases with elevation (orographic lift)
        base_rain = 1200 - pos.x * 1.5  # West is wet
        elev_bonus = max(0, terrain.elevations[i]) * 0.5
        precip[i] = max(100, base_rain + elev_bonus + np.random.uniform(-100, 100))
    
    terrain.fields['precipitation'] = precip
    
    # Carve rivers
    terrain.carve_to_ocean(2)
    
    # Generate soil with our data
    soil = SoilSystem.from_plates(terrain, plates, debug=True)
    
    # Summary stats
    print(f"\nElevation range: {terrain.elevations.min():.0f} to {terrain.elevations.max():.0f}m")
    print(f"Precipitation range: {precip.min():.0f} to {precip.max():.0f}mm")
    
    # Visualize
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.addCoast()
    terrain.hexGrid.builder.adjust("soil", soil.soilOverlay())
    
    return terrain.hexGrid.builder.show()

demoInterestingSoil()
```

**Key changes to get all soil types:**

| Soil Type | Requirement | How we achieve it |
|-----------|-------------|-------------------|
| **Granite** | elev > 1000m, continental | `elevation_scale=8.0` + `formation_type='ridge'` pushes peaks high |
| **Basalt** | oceanic plates | `oceanic_sides=['W','S']` creates clear ocean |
| **Limestone** | mid-elev + precip > 750mm | Added precipitation field, wet on west side |
| **Sandstone** | mid-elev + precip < 750mm | East side is drier (rain shadow) |
| **Alluvial** | coastal lowlands | Rivers carve valleys to coast |

The precipitation gradient simulates prevailing westerly winds hitting mountains — classic rain shadow effect.

In [ ]:
def demoInterestingSoil(seed=42):
    """Create terrain with all 5 soil types represented."""
    mySize = MapSize(600, 600)
    myBounds = MapRect(MapCord(0, 0), mySize)
    
    terrain, plates = Terrain.fromSeeds(
        myBounds,
        radius=12,
        num_plates=8,
        subdivisions=2,
        age='young',
        formation_type='ridge',
        elevation_scale=8.0,
        oceanic_sides=['W', 'S'],
        ocean_fraction=0.3,
        seed=seed
    )
    
    # Add precipitation field - convert hex positions to pixels for x coordinate
    n_hexes = len(terrain.elevations)
    precip = np.zeros(n_hexes)
    center = MapCord(mySize.width / 2, mySize.height / 2)
    
    for i in range(n_hexes):
        pos = terrain.hexGrid.index_to_hexposition(i)
        pixel = pos.pixel(terrain.hexGrid.radius, center)  # Convert to pixel coords
        
        # Rain decreases as you go east (rain shadow)
        base_rain = 1200 - pixel.x * 1.5
        elev_bonus = max(0, terrain.elevations[i]) * 0.5
        precip[i] = max(100, base_rain + elev_bonus + np.random.uniform(-100, 100))
    
    terrain.fields['precipitation'] = precip
    
    # Carve rivers
    rivRivers = terrain.carve_to_ocean(2)
    for i, riv in enumerate(rivRivers):
        riv.carve(20)


    peaks = terrain.find_peaks(8,7)

    rivers = [River.from_peak(terrain, peak) for peak in peaks]
    merged = rivRivers
    rivSVG = ""
    
    #print(StyleCSS.generate(colors))
    # Merge them
    
    # Generate soil with our data
    soil = SoilSystem.from_plates(terrain, plates, debug=True)
    
    # Summary stats
    print(f"\nElevation range: {terrain.elevations.min():.0f} to {terrain.elevations.max():.0f}m")
    print(f"Precipitation range: {precip.min():.0f} to {precip.max():.0f}mm")
    
    # Visualize
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.addCoast()
    terrain.hexGrid.builder.adjust("soil", soil.soilOverlay())

 
    for i, stream in enumerate(merged):
        #print(f"on river {i}")
        if stream is not None:
            rivSVG += stream.svg(styles = [StyleCSS(f"river",stroke="#71b6dbff" , fill = "none",stroke_width=2)])

    
    #terrain.hexGrid.builder.adjust("flows",terrain.flow_diagram())
    #terrain.hexGrid.builder.adjust("rivers", rivSVG)
    #terrain.hexGrid.builder.adjust("legend",drawLegend(legends))
    
    return terrain.hexGrid.builder.show()

demoInterestingSoil()


Done at iter 1: 0 lakes

=== SOIL SYSTEM CREATED ===
Total hexes: 2500
Granite     :   796 hexes ( 31.8%)
Basalt      :  1506 hexes ( 60.2%)
Limestone   :     9 hexes (  0.4%)
Sandstone   :   187 hexes (  7.5%)
Alluvial    :     2 hexes (  0.1%)

Elevation range: -966 to 2020m
Precipitation range: 100 to 1095mm


You aren't using hexpositions correctly. can you fix the demo?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're right! `HexPosition` uses cube coordinates (`q`, `r`, `s`), not `x`. I need to convert to pixel coordinates first:

```python
def demoInterestingSoil(seed=42):
    """Create terrain with all 5 soil types represented."""
    mySize = MapSize(600, 600)
    myBounds = MapRect(MapCord(0, 0), mySize)
    
    terrain, plates = Terrain.fromSeeds(
        myBounds,
        radius=12,
        num_plates=8,
        subdivisions=2,
        age='young',
        formation_type='ridge',
        elevation_scale=8.0,
        oceanic_sides=['W', 'S'],
        ocean_fraction=0.3,
        seed=seed
    )
    
    # Add precipitation field - convert hex positions to pixels for x coordinate
    n_hexes = len(terrain.elevations)
    precip = np.zeros(n_hexes)
    center = MapCord(mySize.width / 2, mySize.height / 2)
    
    for i in range(n_hexes):
        pos = terrain.hexGrid.index_to_hexposition(i)
        pixel = pos.pixel(terrain.hexGrid.radius, center)  # Convert to pixel coords
        
        # Rain decreases as you go east (rain shadow)
        base_rain = 1200 - pixel.x * 1.5
        elev_bonus = max(0, terrain.elevations[i]) * 0.5
        precip[i] = max(100, base_rain + elev_bonus + np.random.uniform(-100, 100))
    
    terrain.fields['precipitation'] = precip
    
    # Carve rivers
    terrain.carve_to_ocean(2)
    
    # Generate soil with our data
    soil = SoilSystem.from_plates(terrain, plates, debug=True)
    
    # Summary stats
    print(f"\nElevation range: {terrain.elevations.min():.0f} to {terrain.elevations.max():.0f}m")
    print(f"Precipitation range: {precip.min():.0f} to {precip.max():.0f}mm")
    
    # Visualize
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.addCoast()
    terrain.hexGrid.builder.adjust("soil", soil.soilOverlay())
    
    return terrain.hexGrid.builder.show()

demoInterestingSoil()
```

The fix: use `pos.pixel(radius, center)` to get a `MapCord` with `.x` and `.y` attributes.

In [ ]:
!ls ../../HexMagic/water/ ../../HexMagic/water . ..

.:
basin.ipynb  erode.ipynb  river.ipynb  soil.ipynb  watershed.ipynb

..:
00_core.ipynb		  07_climate.ipynb	   profile_stats
01_styles.ipynb		  08_hydrology_dup1.ipynb  requirements.txt
02_primitives.ipynb	  09_terraform.ipynb	   sidebar.yml
03_terrain.ipynb	  10_erosion.ipynb	   styles.css
04_voronoi.ipynb	  _quarto.yml		   tmp
05_terrainpatterns.ipynb  index.ipynb		   tutorials
06_river.ipynb		  nbdev.yml		   water
06_weather.ipynb	  plots

../../HexMagic/water:
__init__.py  __pycache__  erode.py  hydrology.py  river.py  soil.py

../../HexMagic/water/:
__init__.py  __pycache__  erode.py  hydrology.py  river.py  soil.py


We are in erode.ipynb. So it is trying to import from from HexMagic.styles import StyleCSS 
but it is in HexMagic/river

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You need to add the parent directory to the path. Add this at the top of your notebook:

```python
import sys
sys.path.insert(0, '../..')
```

Or if you want it to work regardless of where the notebook is run from:

```python
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
```

This lets Python find `HexMagic` when you're working from `HexMagic/water/`.

---

**Note:** Some earlier messages in this dialog were truncated due to context limits, so I may be missing some prior context.